## 补充

* 下载的inception_model文件夹过大，就不上传至github了
* inception系列介绍：https://imlogm.github.io/%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0/inception/
```
GoogLeNet（Inception-v1）：相比AlexNet和VGG，出现了多支路，引入了1×1卷积帮助减少网络计算量
Inception-v2：引入Batch Normalization(BN)；5×5卷积使用两个3×3卷积代替
Inception-v3：n×n卷积分割为1×n和n×1两个卷积
Inception-v4：进一步优化，引入ResNet的shortcut思想
```

In [3]:
import tensorflow as tf
import os
import tarfile
import requests

In [5]:
# inception模型下载
inception_pretrain_model_url = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'
# 模型存放地址
inception_pretrain_model_dir = "inception_model"
if not os.path.exists(inception_pretrain_model_dir): # 不存在文件夹则新建文件夹
    os.makedirs(inception_pretrain_model_dir)

# 获取文件名以及文件路径
filename = inception_pretrain_model_url.split('/')[-1]
filepath = os.path.join(inception_pretrain_model_dir,filename)

# 下载模型
if not os.path.exists(filepath):
    print('download:',filename)
    r = requests.get(inception_pretrain_model_url, stream=True)
    with open(filepath, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
print('finish: ',filename)

#解压文件
tarfile.open(filepath, 'r:gz').extractall(inception_pretrain_model_dir)

log_dir = 'logs/inception_log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# classfy_image_graph_def.pb 为google训练好的模型
inception_graph_def_fiel = os.path.join(inception_pretrain_model_dir,'classify_image_graph_def.pb') # 解压后的文件classify_image_graph_def.pb
with tf.Session() as sess:
    #创建一个图来存放google训练好的模型
    with tf.gfile.FastGFile(inception_graph_def_fiel, 'rb') as f:
        graph_def = tf.GraphDef()
#         graph_def.ParseFromSring(f.read())
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')
    
    # 保存图的结构
    writer = tf.summary.FileWriter(log_dir, sess.graph)
    writer.close()

finish:  inception-2015-12-05.tgz
W0803 11:16:59.486332 10508 deprecation.py:323] From <ipython-input-5-377099d04e15>:33: FastGFile.__init__ (from tensorflow.python.platform.gfile) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.gfile.GFile.
